# Imports

In [1]:
import vertexai
import os
import torch
import json
from vertexai.generative_models import Tool
from langchain.prompts import PromptTemplate
from langchain.tools import Tool
from langchain.globals import set_debug, set_verbose
from langchain.memory import ConversationBufferMemory
import requests
from typing import Dict
from langchain_google_vertexai import HarmBlockThreshold, HarmCategory
from pyowm import OWM
from influxdb_client import InfluxDBClient
from influxdb_client.client.write_api import SYNCHRONOUS
from langchain.tools import tool
from habanero import Crossref
from langchain.agents import initialize_agent, AgentType
from langchain_google_vertexai import VertexAI

In [2]:
from Agents.WeatherAgent import WeatherAgent
from Agents.CrossrefAgent import CrossrefAgent
from Agents.ElsevierAgent import ElsevierAgent
from Agents.DBAgent import DBAgent

# Cache

In [3]:
# Optimize the memory usage

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
torch.cuda.empty_cache()

# API Keys

In [4]:
# Set the API keys for various tools

os.environ["OPENWEATHERMAP_API_KEY"] = "54b1d6c1af466db06bee6219ddb33f7c"
INFLUXDB_URL = "https://apiivm78.etsii.upm.es:8086"
INFLUXDB_TOKEN = "bYNCMsvuiCEoJfPFL5gPgWgDISh79wO4dH9dF_y6cvOKp6uWTRZHtPIwEbRVb2gfFqo3AdygZCQIdbAGBfd31Q=="
INFLUXDB_ORG = "UPM"
INFLUXDB_BUCKET = "LoraWAN"

# Model Set up

In [5]:
vertexai.init(project="summer-surface-443821-r9", location="europe-southwest1")

model = "gemini-1.5-flash"

In [6]:
safety_settings ={
    HarmCategory.HARM_CATEGORY_UNSPECIFIED: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
    }

In [7]:
model_kwargs = {
    # temperature (float): The sampling temperature controls the degree of
    # randomness in token selection.
    "temperature": 0.28,
    # max_output_tokens (int): The token limit determines the maximum amount of
    # text output from one prompt.
    "max_output_tokens": 1000,
    # top_p (float): Tokens are selected from most probable to least until
    # the sum of their probabilities equals the top-p value.
    "top_p": 0.95,
    # top_k (int): The next token is selected from among the top-k most
    # probable tokens.
    "top_k": 40,
    # safety_settings (Dict[HarmCategory, HarmBlockThreshold]): The safety
    # settings to use for generating content.
    "safety_settings": safety_settings,
}

## Memory

In [8]:
# Define memory buffer

memory = ConversationBufferMemory(
        memory_key="chat_history",
        return_messages=True,
    )

/tmp/ipykernel_3930683/2280263513.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


# Templates

In [ ]:
# Define the answer structure

PREFIX = """Answer the following questions as best you can. You have access to the following tools:"""
FORMAT_INSTRUCTIONS = """To use a tool, please use the following format:

```
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
```

When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:

```
Thought: Do I need to use a tool? No
Final Answer: [your response here]
```"""
SUFFIX = """Begin!
Question: {input}
Thought:{agent_scratchpad}"""

In [ ]:
# Define a prompt template

template = "\n\n".join([PREFIX, "{tools}", FORMAT_INSTRUCTIONS, SUFFIX])
prompt = PromptTemplate(template=template, input_variables=["input", "chat_history", "tools", "tool_names", "agent_scratchpad"])
print(prompt)

# Tools

### OpenWeatherMap

In [13]:
from langchain.tools import StructuredTool
from pydantic import BaseModel
import requests
from datetime import datetime

# Define expected input schema
class WeatherInput(BaseModel):
    location: str
    date: str | None = None  # Optional, in "YYYY-MM-DD" format

def resolve_location(location: str) -> tuple[float, float] | None:
    """
    Resolves a location name into latitude and longitude using Open-Meteo's geocoding API.

    Args:
        location (str): The name of the location (e.g., "Madrid" or "New York").

    Returns:
        tuple: A tuple containing latitude and longitude as floats.
               Returns None if the location is not found.
    """
    try:
        response = requests.get(
            "https://geocoding-api.open-meteo.com/v1/search",
            params={"name": location, "count": 1}
        )
        response.raise_for_status()
        data = response.json()
        if "results" in data and len(data["results"]) > 0:
            result = data["results"][0]
            return result["latitude"], result["longitude"]
        else:
            return None
    except Exception as e:
        print(f"Error resolving location: {e}")
        return None

def get_weather(location: str, date: str | None = None) -> str:
    """
    Fetches current or historical weather data for a given location using Open-Meteo.

    Args:
        location (str): The name of the location (e.g., "Madrid" or "New York").
        date (str, optional): A historical date in "YYYY-MM-DD" format. If omitted, current weather is returned.

    Returns:
        str: A string describing the weather conditions for the given location and date.
             If an error occurs or the location is not found, returns an explanatory message.
    """
    coords = resolve_location(location)
    if coords is None:
        return f"Could not resolve the location: {location}."

    lat, lon = coords

    if date:
        # Historical weather
        try:
            response = requests.get(
                "https://archive-api.open-meteo.com/v1/archive",
                params={
                    "latitude": lat,
                    "longitude": lon,
                    "start_date": date,
                    "end_date": date,
                    "hourly": "temperature_2m,relative_humidity_2m",
                    "timezone": "auto"
                }
            )
            response.raise_for_status()
            data = response.json()
            temps = data.get("hourly", {}).get("temperature_2m", [])
            hums = data.get("hourly", {}).get("relative_humidity_2m", [])
            if temps and hums:
                avg_temp = sum(temps) / len(temps)
                avg_hum = sum(hums) / len(hums)
                return (
                    f"On {date}, the average temperature in {location} was {avg_temp:.1f}°C "
                    f"and the average relative humidity was {avg_hum:.1f}%."
                )
            else:
                return f"No weather data available for {location} on {date}."
        except Exception as e:
            return f"Failed to fetch historical weather data: {str(e)}"
    else:
        # Current weather
        try:
            response = requests.get(
                "https://api.open-meteo.com/v1/forecast",
                params={
                    "latitude": lat,
                    "longitude": lon,
                    "current": "temperature_2m,relative_humidity_2m",
                    "timezone": "auto"
                }
            )
            response.raise_for_status()
            data = response.json()
            current = data.get("current", {})
            temp = current.get("temperature_2m")
            hum = current.get("relative_humidity_2m")
            if temp is not None and hum is not None:
                return (
                    f"The current temperature in {location} is {temp}°C with a "
                    f"relative humidity of {hum}%."
                )
            else:
                return f"No current weather data available for {location}."
        except Exception as e:
            return f"Failed to fetch current weather data: {str(e)}"

# Create the tool
weather_tool = StructuredTool.from_function(
    name="weather_tool",
    description=(
        "Fetches current or historical weather data for a specified location. "
        "You can provide a date (YYYY-MM-DD) to retrieve past weather data, "
        "or leave it empty to get current weather."
    ),
    func=get_weather,
    args_schema=WeatherInput,
)


In [14]:
weather_tool.invoke({"location": "Paris"})

'The current temperature in Paris is 22.2°C with a relative humidity of 59%.'

### InfluxDB

In [25]:
from influxdb_client import InfluxDBClient
from langchain.tools import Tool
import re
import json
from pydantic import BaseModel
from langchain.tools.base import StructuredTool

# Client configuration
INFLUXDB_URL = "https://apiivm78.etsii.upm.es:8086"
INFLUXDB_TOKEN = "bYNCMsvuiCEoJfPFL5gPgWgDISh79wO4dH9dF_y6cvOKp6uWTRZHtPIwEbRVb2gfFqo3AdygZCQIdbAGBfd31Q=="
INFLUXDB_ORG = "UPM"
INFLUXDB_BUCKET = "LoraWAN"

client = InfluxDBClient(url=INFLUXDB_URL, token=INFLUXDB_TOKEN, org=INFLUXDB_ORG)
query_api = client.query_api()

# Supported parameters
VALID_METRICS = {"temperature", "humidity", "light", "motion", "vdd"}
VALID_AGGREGATIONS = {"mean", "max", "min", "sum"}

# Define expected input schema for the tool using Pydantic
class InfluxDBQueryInput(BaseModel):
    """
    Defines the input parameters required for querying InfluxDB.
    """
    metric: str          # Sensor metric to query (e.g., temperature, humidity)
    time_range: str      # Time range for the query (e.g., 24h, 7d)
    aggregation: str     # Aggregation function (e.g., mean, max)

# Function to construct Flux query dynamically
def construct_flux_query(params: dict) -> str:
    """
    Constructs a Flux query based on extracted parameters.
    
    Args:
        params (dict): A dictionary containing 'metric', 'time_range', and 'aggregation'.
    
    Returns:
        str: A formatted Flux query.
    """
    measurement = "sensor_data"  # Default measurement
    field = params.get("metric", "humidity")  # Default metric
    time_range = params.get("time_range", "24h")  # Default to 24h if missing
    aggregation = params.get("aggregation", "mean")  # Default to mean

    # Validate metric
    if field not in VALID_METRICS:
        raise ValueError(f"❌ Invalid metric '{field}'. Available metrics: {', '.join(VALID_METRICS)}")

    # Validate aggregation function
    if aggregation not in VALID_AGGREGATIONS:
        raise ValueError(f"❌ Invalid aggregation '{aggregation}'. Available functions: {', '.join(VALID_AGGREGATIONS)}")

    # Construct Flux query
    flux_query = f"""
    from(bucket: "{INFLUXDB_BUCKET}")
      |> range(start: -{time_range})
      |> filter(fn: (r) => r["_measurement"] == "{measurement}")
      |> filter(fn: (r) => r["_field"] == "{field}")
      |> aggregateWindow(every: 1h, fn: {aggregation}, createEmpty: false)
      |> yield(name: "result")
    """
    return flux_query

# Function that receives individual arguments (required by StructuredTool)
def query_influxdb(metric: str, time_range: str, aggregation: str) -> str:
    """
    StructuredTool-compatible function to query InfluxDB using individual parameters.

    Args:
        metric (str): Sensor metric to query (e.g., "temperature", "humidity").
        time_range (str): Time range for the query (e.g., "24h", "7d").
        aggregation (str): Aggregation function to apply (e.g., "mean", "max").

    Returns:
        str: Formatted result or error message.
    """
    params = {
        "metric": metric,
        "time_range": time_range,
        "aggregation": aggregation
    }
    return _query_influxdb_internal(params)

# Internal function to perform the actual query logic
def _query_influxdb_internal(params: dict) -> str:
    """
    Constructs and executes a Flux query on InfluxDB from parameter dictionary.

    Args:
        params (dict): Dictionary containing 'metric', 'time_range', and 'aggregation'.

    Returns:
        str: Query results or an error message.
    """
    try:
        # Build the Flux query dynamically from parameters
        flux_query = construct_flux_query(params)

        print(f"📊 Extracted Parameters: {params}")
        print(f"🔥 Executing Flux Query:\n{flux_query}") 

        # Execute the query using InfluxDB client
        result = query_api.query(org=INFLUXDB_ORG, query=flux_query)
        results = []

        # Format the results
        for table in result:
            for record in table.records:
                results.append(f"Time: {record.get_time()}, Value: {record.get_value()}")

        return "\n".join(results) if results else "⚠️ No data found in the database. Verify if data exists for this time range."
    
    except ValueError as ve:
        return str(ve)  # Return validation error messages

    except Exception as e:
        return f"❌ Error querying InfluxDB: {str(e)}"

# Function to extract the time range from a query
def extract_time_range(user_query: str) -> str:
    """
    Extracts the time range from a user query.

    Args:
        user_query (str): The input query from the user.

    Returns:
        str: A formatted time range for InfluxDB (e.g., "24h", "7d", "30d").
    """
    time_patterns = {
        r"(\d+)\s*(minute|minutes|min)": "m",
        r"(\d+)\s*(hour|hours|h)": "h",
        r"(\d+)\s*(day|days|d)": "d",
        r"(\d+)\s*(week|weeks|w)": "w",
        r"(\d+)\s*(month|months|mo)": "d",  # Approximate: 1 month = 30 days
        r"(\d+)\s*(year|years|y)": "d"  # Approximate: 1 year = 365 days
    }

    detected_range = "24h"  # Default if no time range is found

    for pattern, unit in time_patterns.items():
        match = re.search(pattern, user_query, re.IGNORECASE)
        if match:
            value = int(match.group(1))  # Extract numeric value
            if "month" in pattern:
                value *= 30  # Convert months to days
            elif "year" in pattern:
                value *= 365  # Convert years to days
            detected_range = f"{value}{unit}"
            break

    return detected_range

# LangChain compatible tool

influx_tool = StructuredTool.from_function(
    name="InfluxDB Query Tool",
    description=(
        "Fetches sensor data from InfluxDB. "
        "Requires parameters like metric (e.g., humidity, temperature), "
        "time_range (e.g., 24h), and aggregation (e.g., mean)."
    ),
    func=query_influxdb,
    args_schema=InfluxDBQueryInput
)

In [26]:
result = query_influxdb(metric="temperature", time_range="100d", aggregation="mean")
print(result)

📊 Extracted Parameters: {'metric': 'temperature', 'time_range': '100d', 'aggregation': 'mean'}
🔥 Executing Flux Query:

    from(bucket: "LoraWAN")
      |> range(start: -100d)
      |> filter(fn: (r) => r["_measurement"] == "sensor_data")
      |> filter(fn: (r) => r["_field"] == "temperature")
      |> aggregateWindow(every: 1h, fn: mean, createEmpty: false)
      |> yield(name: "result")
    
Time: 2025-01-13 16:00:00+00:00, Value: 12.1
Time: 2025-01-13 17:00:00+00:00, Value: 12.124999999999998
Time: 2025-01-13 18:00:00+00:00, Value: 12.091666666666663
Time: 2025-01-13 19:00:00+00:00, Value: 12.066666666666665
Time: 2025-01-13 20:00:00+00:00, Value: 12.033333333333331
Time: 2025-01-13 21:00:00+00:00, Value: 12.0
Time: 2025-01-13 22:00:00+00:00, Value: 11.975
Time: 2025-01-13 23:00:00+00:00, Value: 11.925000000000002
Time: 2025-01-14 00:00:00+00:00, Value: 11.900000000000004
Time: 2025-01-14 01:00:00+00:00, Value: 11.900000000000004
Time: 2025-01-14 02:00:00+00:00, Value: 11.90000000

### Elsevier

In [ ]:
def get_article_content(title: str) -> str:
    """
    Fetch the content of a specific article using Elsevier's APIs based on its title.

    Args:
    - title (str): The title of the article to search for.

    Returns:
    - str: The content or abstract of the specified article.
    """

    API_KEY = "87ab69edd16f0cdb92e611b99b8f4ee6"
    BASE_SEARCH_URL = "https://api.elsevier.com/content/search/scopus"
    BASE_ARTICLE_URL = "https://api.elsevier.com/content/article/doi"
    BASE_ARTICLE_EID_URL = "https://api.elsevier.com/content/article/eid"

    # 1. Search article by title
    headers = {
        "Accept": "application/json",
        "X-ELS-APIKey": API_KEY,
    }
    params = {
        "query": f"TITLE({title})",
        "count": 1,
    }
    
    search_response = requests.get(BASE_SEARCH_URL, headers=headers, params=params)

    if search_response.status_code != 200:
        return f"Error: No se pudo buscar el artículo. Código de estado: {search_response.status_code}, Error: {search_response.text}"

    search_data = search_response.json()
    entries = search_data.get("search-results", {}).get("entry", [])

    if not entries:
        return f"No se encontró ningún artículo con el título '{title}'."

    # 2. Fetch article identifier
    article_entry = entries[0]
    print("Artículo encontrado:", article_entry)  # Debugging info

    doi = article_entry.get("prism:doi")
    eid = article_entry.get("eid")

    if not doi and not eid:
        return f"No se encontró DOI ni EID para el artículo con el título '{title}'. No se puede recuperar el contenido."

    # 3. Fetch article content using DOI
    if doi:
        article_url = f"{BASE_ARTICLE_URL}/{doi}"
    else:
        article_url = f"{BASE_ARTICLE_EID_URL}/{eid}"

    print(f"Intentando recuperar el artículo desde: {article_url}")  # Debugging info

    article_response = requests.get(article_url, headers=headers)

    if article_response.status_code != 200:
        return f"Error: No se pudo recuperar el contenido del artículo. Código de estado: {article_response.status_code}, Error: {article_response.text}"

    article_data = article_response.json()

    abstract = article_data.get("full-text-retrieval-response", {}).get("coredata", {}).get("dc:description", "No abstract found")

    return f"Contenido del artículo '{title}':\n\n{abstract}"


In [ ]:
title = "Taking the next step with generative artificial intelligence: The transformative role of multimodal large language models in science education"
content = get_article_content(title)
print(content)

### Crossref

In [ ]:
def crossref(subject: str) -> Dict:
    limit = 5
    results = {}
    cr = Crossref()
    try:
        result = cr.works(query=subject, limit=limit)
        for i in range(0, limit-1):
            title = result['message']['items'][i].get('title', ['No Title'])[0]
            abstract = result['message']['items'][i].get('abstract', 'No abstract available')
            results[title] = abstract
        return results
    except Exception as e:
        return {"error": str(e)}

crossref_tool = Tool(
    name="Crossref article search",
    func=crossref,
    description="Use this to fetch article titles and abstracts based on a given subject.",
)


In [ ]:
cr = Crossref()
x = cr.works(query = "ecology")
x['message']['items'][4]['abstract']

In [ ]:
a = crossref(subject="computing")

print(a)

# Agents

## Weather Agent

In [8]:
weather_agent = WeatherAgent()

In [13]:
weather_agent.run("What was the weather like in MAdrid the 6th of april of 2025 at 11am")

"I can get you the weather for Madrid on April 6, 2025, but I can't specify the time. Would you like the weather for the whole day?\n"

## DB Agent 

In [14]:
from Agents.DBAgent import DBAgent
from langchain_core.messages import HumanMessage

agent = DBAgent()

/mnt/sdb1/home/amartinez/MARTINEZ_Alex_Llm-Agents_2024/src/tfg/Agents/DBAgent.py:34: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory=ConversationBufferMemory()
/mnt/sdb1/home/amartinez/MARTINEZ_Alex_Llm-Agents_2024/src/tfg/Agents/DBAgent.py:29: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  self.agent_executor = initi

In [22]:
response = agent.invoke({"messages": [HumanMessage(content="What is the max temperature of the last 42 days?")]})

🧠 db_agent received query: What is the max temperature of the last 42 days?


> Entering new AgentExecutor chain...
```json
{
  "action": "InfluxDB Query Tool",
  "action_input": {
    "metric": "temperature",
    "time_range": "42d",
    "aggregation": "max"
  }
}
```📊 Extracted Parameters: {'metric': 'temperature', 'time_range': '42d', 'aggregation': 'max'}
🔥 Executing Flux Query:

    from(bucket: "LoraWAN")
      |> range(start: -42d)
      |> filter(fn: (r) => r["_measurement"] == "sensor_data")
      |> filter(fn: (r) => r["_field"] == "temperature")
      |> aggregateWindow(every: 1h, fn: max, createEmpty: false)
      |> yield(name: "result")
    

Observation: Time: 2025-03-22 15:00:00+00:00, Value: 15.0
Time: 2025-03-22 16:00:00+00:00, Value: 15.0
Time: 2025-03-22 17:00:00+00:00, Value: 15.0
Time: 2025-03-22 18:00:00+00:00, Value: 15.0
Time: 2025-03-22 19:00:00+00:00, Value: 15.0
Time: 2025-03-22 20:00:00+00:00, Value: 14.9
Time: 2025-03-22 21:00:00+00:00, Value: 14.8
Time: 2

In [11]:
agent.run("What was the max humidity of the last 20 days") 

AttributeError: 'DBAgent' object has no attribute 'run'

## Elsevier Agent

In [12]:
elsevier_agent = ElsevierAgent()

In [16]:
elsevier_agent.run("Search the article AI Open")

'I am sorry, I cannot fulfill this request. I am unable to retrieve the article content. The article was not found.\n'

## Crossref Agent

In [14]:
crossref_agent = CrossrefAgent()

In [15]:
crossref_agent.run("I want to know more about quantum computing, can you help me?")

'I found these articles on quantum computing:\n\n"Quantum Computing I" - Abstract: A remarkable application of quantum mechanical concepts of coherent superposition and quantum entanglement is a quantum computer which can solve certain problems at speeds unbelievably faster than the conventional computer. In this chapter, the basic principles and the conditions for the implementation of the quantum computer are introduced and the limitations imposed by the probabilistic nature of quantum mechanics and the inevitable decoherence phenomenon are discussed. Next the basic building blocks, the quantum logic gates, are introduced. These include the Hadamard, the CNOT, and the quantum phase gates. After these preliminaries, the implementation of the Deutsch algorithm, quantum teleportation, and quantum dense coding in terms of the quantum logic gates are discussed. It is also shown how the Bell states can be produced and measured using a sequence of quantum logic gates.\n\n"Quantum Computing 

In [ ]:
# Agents dictionary
agents = {
    "weather": weather_agent,
    "article title": crossref_agent,
    "article content": elsevier_agent,
    "generic": weather_agent,  
}

# Agent Router

In [ ]:
# Initialize the router
router = CustomAgentRouter()

In [ ]:
inputs = [
    "What is the weather like in Madrid?",  # Should go to WeatherAgent
    "Search for articles about machine learning.",  # Should go to CrossrefAgent
    "I need the content of an article titled 'Deep Learning Basics'."  # Should go to ElsevierAgent
]

In [ ]:
from transformers import pipeline
pipeline = pipeline("text-classification", model="distilbert-base-uncased", top_k=None)

In [ ]:
predictions = pipeline("What is the weather like in Madrid?")
print("Predictions output:", predictions)

In [ ]:
response = router.run( "What is the weather like in Madrid?")
print(f"Response: {response}\n")

In [ ]:
for input_text in inputs:
    print(f"Input: {input_text}")
    response = router.run(input_text)
    print(f"Response: {response}\n")

# Tests

In [3]:
from langgraph.graph import StateGraph, START, END
from typing import Dict, Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages
from langchain.memory import ConversationBufferMemory
from CustomAgentRouter import CustomAgentRouter
from Agents.WeatherAgent import WeatherAgent
from Agents.DBAgent import DBAgent
from Agents.CrossrefAgent import CrossrefAgent
from Agents.ElsevierAgent import ElsevierAgent
import transformers
from langchain.schema import HumanMessage
transformers.logging.set_verbosity_error()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Define the state structure with memory
class State(TypedDict):
    messages: Annotated[list, add_messages]
    route: str
    memory: ConversationBufferMemory

def classify_query(state: State) -> Dict[str, str]:
    """
    Uses CustomAgentRouter to classify user input and determine the correct agent.
    """
    user_input = state["messages"][-1].content
    router = CustomAgentRouter()
    category, _ = router.classify_text(user_input)
    return {"route": category}

def route_decision(state: State) -> str:
    """
    Determines the next node based on classification result.
    """
    valid_routes = {"weather", "database", "article title", "article content"}
    return state["route"] if state["route"] in valid_routes else "generic"

# Agent functions with memory integration
def weather_agent(state: State) -> Dict[str, str]:
    user_input = state["messages"][-1].content
    agent = WeatherAgent()
    response = agent.run(user_input)
    state["memory"].save_context({"user_input": user_input}, {"response": response})
    return {"messages": state["messages"] + [{"role": "assistant", "content": response}]}

def db_agent(state: State) -> Dict[str, str]:
    user_input = state["messages"][-1].content
    agent = DBAgent()
    response = agent.run(user_input)
    state["memory"].save_context({"user_input": user_input}, {"response": response})
    return {"messages": state["messages"] + [{"role": "assistant", "content": response}]}

def crossref_agent(state: State) -> Dict[str, str]:
    user_input = state["messages"][-1].content
    agent = CrossrefAgent()
    response = agent.run(user_input)
    state["memory"].save_context({"user_input": user_input}, {"response": response})
    return {"messages": state["messages"] + [{"role": "assistant", "content": response}]}

def elsevier_agent(state: State) -> Dict[str, str]:
    user_input = state["messages"][-1].content
    agent = ElsevierAgent()
    response = agent.run(user_input)
    state["memory"].save_context({"user_input": user_input}, {"response": response})
    return {"messages": state["messages"] + [{"role": "assistant", "content": response}]}

# Define the LangGraph workflow
graph = StateGraph(State)

graph.add_node("classify", classify_query)
graph.add_node("weather", weather_agent)
graph.add_node("database", db_agent)
graph.add_node("crossref", crossref_agent)
graph.add_node("elsevier", elsevier_agent)
graph.add_node("generic", lambda state: {"messages": state["messages"] + [{"role": "assistant", "content": "I'm not sure how to help with that."}]})

graph.add_edge(START, "classify")
graph.add_conditional_edges("classify", route_decision, {
    "weather": "weather",
    "database": "database",
    "article title": "crossref",
    "article content": "elsevier",
    "generic": "generic"
})

graph.add_edge("weather", END)
graph.add_edge("database", END)
graph.add_edge("crossref", END)
graph.add_edge("elsevier", END)
graph.add_edge("generic", END)

compiled_graph = graph.compile()

def run_conversation(user_input: str):
    state = {
        "messages": [{"role": "user", "content": user_input}],
        "memory": ConversationBufferMemory()
    }
    for event in compiled_graph.stream(state):
        for value in event.values():
            if "messages" in value:
                print("Assistant:", value["messages"][-1]["content"])
            else:
                print("Assistant: (No response received)")

In [ ]:
while True:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break

        stream_graph_updates(user_input)